In [25]:
import pandas as pd
import numpy as np
import os
from io import StringIO

from google.cloud import storage
from google.cloud import pubsub_v1
import logging
from concurrent.futures import TimeoutError
import json

os.getcwd()

'C:\\Users\\BOURGET Rick'

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'C:/Users/BOURGET Rick/Documents/pipelinePOC/our-shield-373717-9ba7bcbe7650.json'

In [14]:
message = "IKUSO"
isinstance({'test_values':message.encode("utf-8")},str)

False

In [29]:
bucket_name = "spark_bucket_rbgt"
file = storage.Client(project="our-shield-373717").bucket(bucket_name).blob("customer-orders.csv").download_as_string()
file = file.decode("utf-8")
file = StringIO(file)
file = pd.read_csv(file, header=None)
file

,0,1,2
0,44,8602,37.19
1,35,5368,65.89
2,2,3391,40.64
3,47,6694,14.98
4,29,680,13.08
...,...,...,...
9995,61,229,86.69
9996,50,4331,92.79
9997,2,9155,56.05
9998,23,2477,14.89


In [31]:
file[0].values

array([44, 35,  2, ...,  2, 23, 97], dtype=int64)

In [34]:
file.loc[0,:]

0      44.00
1    8602.00
2      37.19
Name: 0, dtype: float64

### Collecter la data depuis Cloud Storage

In [38]:
bucket_name = "spark_bucket_rbgt"
# On télécharge le csv et on le formatte pour pouvoir le lire avec Pandas
file = storage.Client(project="our-shield-373717").bucket(bucket_name).blob("customer-orders.csv").download_as_string()
file = file.decode("utf-8")
file = StringIO(file)
file = pd.read_csv(file, header=None)


publisher = pubsub_v1.PublisherClient()
topic_name = "projects/our-shield-373717/topics/collect-data"

for i in range(len(file)):
    row = file.loc[i,:]
    data = {'customer_id': row[0],'order_id': row[1],'order_amount':row[2]}

    data = json.dumps(data)
    data = data.encode("utf-8")

    future = publisher.publish(topic_name, data=data)